In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Función para preprocesar los datos
def preprocess_data(data):
    # Convertir DAYS_EMPLOYED a positivo
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].abs()

    # Asignar 0 a CAR_OWN_AGE donde FLAG_OWN_CAR es 'N'
    data.loc[data['FLAG_OWN_CAR'] == 'N', 'CAR_OWN_AGE'] = 0

    # Rellenar los valores faltantes
    num_columns = ['CAR_OWN_AGE', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_EMPLOYED',
                   'CNT_FAM_MEMBERS', 'BASEMENTAREA_AVG', 'YEARS_BUILD_AVG',
                   'COMMONAREA_AVG', 'FLOORSMAX_AVG']
    cat_columns = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'OCCUPATION_TYPE']

    for col in num_columns:
        data[col].fillna(0, inplace=True)

    for col in cat_columns:
        data[col].fillna('No aplica', inplace=True)

    return data[num_columns + cat_columns], num_columns, cat_columns

# Cargando el dataset de entrenamiento y preprocesamiento
dataset_train = pd.read_csv('/content/sample_data/datasetv2train.csv')
X_train, num_columns, cat_columns = preprocess_data(dataset_train)
y_train = dataset_train['target']

# Preprocesador y modelo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_columns)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=0))
])

# Entrenamiento del modelo
pipeline.fit(X_train, y_train)

# Cargando el dataset de predicciones y preprocesamiento
dataset_pred = pd.read_csv('/content/sample_data/datasetv2.csv')
X_pred, _, _ = preprocess_data(dataset_pred)

# Haciendo predicciones
predictions = pipeline.predict(X_pred)

# Añadiendo las predicciones al dataset
dataset_pred['target'] = predictions

# Guardando las predicciones en un nuevo archivo CSV
dataset_pred.to_csv('datasetv2_with_predictions.csv', index=False)

print("Predicciones completadas y guardadas en 'datasetv2_with_predictions.csv'")

# Gráfica de las predicciones
plt.figure(figsize=(10, 6))

# Contar la cantidad de predicciones para cada clase
sns.countplot(x='target', data=dataset_pred)

plt.title('Distribución de las predicciones')
plt.xlabel('Clase')
plt.ylabel('Cantidad')
plt.xticks(ticks=[0,1], labels=['No Aprobado (0)', 'Aprobado (1)'])

plt.show()

####################################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir el conjunto de datos original en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    dataset_train.drop('target', axis=1),  # Asumiendo que 'target' es la columna objetivo
    dataset_train['target'],
    test_size=0.2,  # Porcentaje del conjunto de datos a usar como prueba
    random_state=0
)

# Preprocesar los conjuntos de entrenamiento y prueba
X_train_processed, num_columns, cat_columns = preprocess_data(X_train)
X_test_processed, _, _ = preprocess_data(X_test)

# Reentrenar el modelo con los mejores hiperparámetros encontrados
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=0))
])

best_pipeline.fit(X_train_processed, y_train)

# Evaluar en el conjunto de entrenamiento
train_predictions = best_pipeline.predict(X_train_processed)
train_accuracy = accuracy_score(y_train, train_predictions)
print("Precisión en el conjunto de entrenamiento:", train_accuracy)

# Evaluar en el conjunto de prueba
test_predictions = best_pipeline.predict(X_test_processed)
test_accuracy = accuracy_score(y_test, test_predictions)
print("Precisión en el conjunto de prueba:", test_accuracy)

# Diagnóstico de overfitting o bias
if train_accuracy > test_accuracy:
    if train_accuracy - test_accuracy > 0.1:  # Umbral de diferencia
        print("Posible overfitting.")
    else:
        print("Diferencia aceptable, no hay signos claros de overfitting.")
else:
    print("El modelo puede tener un alto sesgo.")

# Visualización de las métricas de rendimiento
plt.bar(['Entrenamiento', 'Prueba'], [train_accuracy, test_accuracy], color=['blue', 'orange'])
plt.xlabel('Conjunto de datos')
plt.ylabel('Precisión')
plt.title('Comparación de la precisión del modelo')
plt.show()

####################################


import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns



# Cargando el dataset
dataset = pd.read_csv('/content/sample_data/datasetv2.csv')
X, num_columns, cat_columns = preprocess_data(dataset)

# Preprocesador para escalado y codificación one-hot
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_columns)
    ]
)

# Crear un pipeline con el preprocesador
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Preprocesar los datos
X_processed = pipeline.fit_transform(X)

# Definir el modelo K-means con un número arbitrario de clusters
kmeans = KMeans(n_clusters=2, random_state=0)

# Ajustar el modelo a los datos preprocesados
kmeans.fit(X_processed)

# Etiquetas de cluster
clusters = kmeans.labels_

# Añadir las etiquetas de cluster al dataframe original para un análisis más detallado
dataset['Cluster'] = clusters

# Guardar el dataframe con las etiquetas de cluster en un archivo CSV
dataset.to_csv('datasetv2_with_clusters.csv', index=False)

# Visualizar la distribución de los clusters
plt.figure(figsize=(8, 6))
sns.countplot(x='Cluster', data=dataset)
plt.title('Distribución de los Clusters')
plt.xlabel('Cluster')
plt.ylabel('Cantidad')
plt.show()

print("Clustering completado y guardado en 'datasetv2_with_clusters.csv'")




FileNotFoundError: ignored